In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Module 2. Векторная модель текста и классификация длинных текстов ###

**2.3.1**

In [ ]:
from sympy import diff, symbols, log, exp
x, y, w, b = symbols('x y w b')
y_hat = 1/(1+exp(-w*x-b))
eq = -y*log(y_hat)-(1-y)*log(1-y_hat)
print(diff(eq, w).simplify())

In [ ]:
import sympy.parsing.sympy_parser

sample_expr_str = 'x*(-y*exp(-b - w*x) - y + 1)/(exp(-b - w*x) + 1)'
sample_expr = sympy.parsing.sympy_parser.parse_expr(sample_expr_str)
sample_value = sample_expr.evalf(subs=dict(x=0.5, y=1, w=4, b=1))
print(sample_value)

**2.3.2**

In [ ]:
from sympy import diff, symbols, log, exp
x, y, w, b = symbols('x y w b')
y_hat = 1/(1+exp(-w*x-b))
eq = -y*log(y_hat)-(1-y)*log(1-y_hat)
print(diff(eq, b).simplify())

**2.3.3**

In [ ]:
from sympy import diff, symbols, log, exp
x, y, w, b, c = symbols('x y w b, c')
y_hat = 1/(1+exp(-w*x-b))
reg = c * (w**2 + b**2)
eq = -y*log(y_hat)-(1-y)*log(1-y_hat) + reg
print(diff(eq, w).simplify())

**2.3.6**

In [ ]:
import sys
import numpy as np

def parse_array(s):
    return np.array([int(s.strip()) for s in s.strip().split(' ')])

def read_array(inp):
    return parse_array(inp)

def calculate_pmi(a, b):
    #p(a,b) - вероятность того, что оба события совершились одновременно (и там и там единица).
    #p(a) , p(b) - вероятности реализации случайных событий.
    #pmi(a,b) = log(p(a,b)/(p(a)*p(b)))
    p_a_b = np.mean(a * b)
    p_a = np.mean(a)
    p_b = np.mean(b)
    return np.log(p_a_b / (p_a * p_b))  

a = read_array('1 0 0 1 1 0')
b = read_array('1 0 0 0 1 0')
pmi_value = calculate_pmi(a, b)

print('{:.6f}'.format(pmi_value))

**2.3.7**

In [ ]:
import numpy as np
s = 2
P = 10 / 10000 #заданная плотность распределения
Z = np.array([i**(-2) for i in range(1, 1001)]).sum()
rank = (1 / (Z * 10 / 10000))**(1/s)
num = int(1000 - rank) + 1
num

**2.4.3**

In [ ]:
import re
import sys

text = [
'Контактный телефон: 123123.',
'Что-нибудь надо придумать.',
'Значение числа Е=2.7182.',
'Демон123, как тебя зовут в реале?',
'-1-.15=-1.15',
'- 1 - .15 = -1.15',
'Какого ;%:?* тут происходит?'
]

# модифицируйте это регулярное выражение
parts = [
    r'[а-яё]+',
    r'-?\d*[.,]?\d+',
    r'\S'
]
#re.I -  re.IGNORECASE - Perform case-insensitive matching
#Флаг, избавляет нас от необходимости указывать заглавные буквы алфавита 
#при выделении слов и достаточно указать лишь диапазон строчных букв.
TOKENIZE_RE = re.compile(r'|'.join(parts), re.I) 


def tokenize(txt):
    return TOKENIZE_RE.findall(txt)


for line in text:
    print(' '.join(tokenize(line.strip().lower())))

**2.4.5**

In [ ]:
import re
import sys
import collections
import numpy as np



def tokenize_text(text, lower=True):
    parts = [
        r'[\w\d]+'
    ]
    TOKENIZE_RE = re.compile(r'|'.join(parts), re.I)
    if lower:
        return TOKENIZE_RE.findall(text.lower())
    else:
        return TOKENIZE_RE.findall(text)

def tokenize_corpus(texts, lower=True):
    return [tokenize_text(text, lower=lower) for text in texts]

def build_vocabulary(tokenize_texts):
    word_counts = collections.defaultdict(int)
    
    for text in tokenize_texts:
        unique_tokens = set(text)
        for token in unique_tokens:
            word_counts[token] += 1
    doc_count = len(tokenize_texts)
    
    word_counts_sorted = sorted(word_counts.items(), reverse=False, key=lambda kv:(kv[1], kv[0]))
    
    word2id = {word: i for i, (word, _) in enumerate(word_counts_sorted)}
    word2freq = np.array([word_count/doc_count for (_, word_count) in word_counts_sorted], dtype='float32')
    
    return word2id, word2freq

texts = [
"Казнить нельзя, помиловать. Нельзя наказывать.",
"Казнить, нельзя помиловать. Нельзя освободить.",
"Нельзя не помиловать.",
"Обязательно освободить."
]

tokenized_texts = tokenize_corpus(texts)
voc_word2id, voc_word2freq = build_vocabulary(tokenized_texts)

print(' '.join(voc_word2id.keys()))
print(' '.join(map(str, voc_word2id.values())))
print(' '.join(map(str, voc_word2freq)))

In [ ]:
# Федор Краснов
# 2 years ago
# Привет,
# для любопытных: с помощью sklearn тоже можно сделать, но есть особенность, которой и делюсь.

corpus = [
    'Казнить нельзя, помиловать. Нельзя наказывать.',
    'Казнить, нельзя помиловать. Нельзя освободить.',
    'Нельзя не помиловать.',
    'Обязательно освободить.']

## Получение df
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(smooth_idf=False, use_idf=True)
vectorizer.fit_transform(corpus)
word_doc_freq = 1/np.exp(vectorizer.idf_ - 1) ## ВОТ ОСОБЕННОСТЬ ПЕРЕВОДА ИЗ IDF В DF

# Словарь тут: 
# vectorizer.get_feature_names()

**2.4.7**

In [ ]:
import scipy.sparse
import numpy as np
def vectorize_texts_ltfidf(tokenized_texts, voc_word2id, voc_word2freq):
    result = scipy.sparse.dok_matrix((len(tokenized_texts), len(voc_word2id)), dtype='float32')
    for text_i, text in enumerate(tokenized_texts):
        for token in text:
            if token in voc_word2id.keys():
                result[text_i, voc_word2id[token]] += 1
    result = result.tocsr() #переводим матрицу в режим быстрой работы со строками (это очень важно!!)
    result = result.multiply(1/result.sum(1))
    result = scipy.sparse.dok_matrix(np.log(result.toarray()+1))
    result = result.multiply(1 / voc_word2freq)
    feats_std = result.toarray().std(0, ddof=1)
    feats_mean = result.toarray().mean(0)
    return (result.toarray() - feats_mean) / feats_std

In [ ]:
vectorize_texts = vectorize_texts_ltfidf(tokenized_texts, voc_word2id, voc_word2freq)
[print(' '.join(map(str, row))) for row in vectorize_texts];

In [ ]:
# Salavat Garifullin
from collections import Counter
import pandas as pd
import re

corpus = ['Казнить нельзя, помиловать. Нельзя наказывать.',
         'Казнить, нельзя помиловать. Нельзя освободить.',
         'Нельзя не помиловать.',
         'Обязательно освободить.']
TOKENIZE_RE = re.compile(r'[\w\d]+')

corpus = pd.Series(corpus).str.lower().apply(TOKENIZE_RE.findall)
corpus_counts = corpus.apply(Counter).apply(pd.Series).fillna(0).astype(int)
word2freq = (corpus_counts > 0).mean(0).sort_index().sort_values()

### ^ код из 5 шага

import numpy as np

TF = (corpus_counts[word2freq.index].T / corpus.apply(len)).T
lTFIDF = np.log(TF + 1) / word2freq
feature_matrix  = (lTFIDF - lTFIDF.mean()) / lTFIDF.std(0, ddof=1)

for doc in feature_matrix.round(2).values:
    print(*doc)

**2.4 Final task**

In [ ]:
ДЗ по семинару реализовано в отдельном ноутбуке: stepik-nn-and-nlp 2.4.final (bag-of-words-models).ipynb

### Module 3. Базовые нейросетевые методы для работы с текстами ###

**3.3 Final task**

In [ ]:
ДЗ по семинару реализовано в отдельном ноутбуке: stepik-nn-and-nlp 3.3 final (word-embeddings).ipynb

**3.4.1**

In [ ]:
import sys
import ast
import numpy as np


def parse_array(s):
    return np.array(ast.literal_eval(s))

def read_array():
    return parse_array(sys.stdin.readline())

def write_array(arr):
    print(repr(arr.tolist()))


def generate_w2v_sgns_samples(text, window_size, vocab_size, ns_rate):
    """
    text - list of integer numbers - ids of tokens in text
    window_size - odd integer - width of window
    vocab_size - positive integer - number of tokens in vocabulary
    ns_rate - positive integer - number of negative tokens to sample per one positive sample

    returns list of training samples (CenterWord, CtxWord, Label)
    """
    rwindow = window_size // 2
    output = []
    for i in range(len(text)):
        pos_sample = [*text[max(i-rwindow,0):i], *text[i+1:i+rwindow+1]]
        output.extend([[text[i], pos_id, 1] for pos_id in pos_sample])
        output.extend([[text[i], neg_id, 0] for neg_id in np.random.randint(vocab_size, size=ns_rate*len(pos_sample))])
    return output

text = np.random.randint(100,size=100)
window_size = 7
vocab_size = 100
ns_rate = 3

result = generate_w2v_sgns_samples(text, window_size, vocab_size, ns_rate)

#write_array(np.array(result))

**3.4.2**

In [ ]:
def update_w2v_weights(center_embeddings, context_embeddings, center_word, context_word, label, learning_rate):
    """
    center_embeddings - VocabSize x EmbSize
    context_embeddings - VocabSize x EmbSize
    center_word - int - identifier of center word
    context_word - int - identifier of context word
    label - 1 if context_word is real, 0 if it is negative
    learning_rate - float > 0 - size of gradient step
    """
    #здесь для истории, заменил свой код (которым решил задание), 
    #кодом Эдуарда Смирнова, т.к. он более читабельный (сама суть аналогична)
    w = center_embeddings[center_word]
    d = context_embeddings[context_word]

    sigma = 1/(1+np.exp(-w@d))
    grad_w = (sigma-label) * d
    grad_d = (sigma-label) * w

    w -= learning_rate * grad_w
    d -= learning_rate * grad_d

**3.4.3**

In [ ]:
import sys
import ast
import numpy as np


def read_list():
    return ast.literal_eval(sys.stdin.readline())

def parse_array(s):
    return np.array(ast.literal_eval(s))

def read_array():
    return parse_array(sys.stdin.readline())

def write_array(arr):
    print(repr(arr.tolist()))


def generate_ft_sgns_samples(text, window_size, vocab_size, ns_rate, token2subwords):
    """
    text - list of integer numbers - ids of tokens in text
    window_size - odd integer - width of window
    vocab_size - positive integer - number of tokens in vocabulary
    ns_rate - positive integer - number of negative tokens to sample per one positive sample
    token2subwords - list of lists of int - i-th sublist contains list of identifiers of n-grams for token #i (list of subword units)

    returns list of training samples (CenterSubwords, CtxWord, Label)
    """
    rwindow = window_size // 2
    output = []
    for i in range(len(text)):
        pos_sample = [*text[max(i-rwindow,0):i], *text[i+1:i+rwindow+1]]
        output.extend([[[text[i], *token2subwords[text[i]]], pos_id, 1] for pos_id in pos_sample])
        output.extend([[[text[i], *token2subwords[text[i]]], neg_id, 0] for neg_id in np.random.randint(vocab_size, size=ns_rate*len(pos_sample))])
    return output


text = [1, 2, 0, 1, 4, 0, 4, 1, 5, 4, 5, 4, 5, 1]
window_size = 3
vocab_size = 6
ns_rate = 2
token2subwords = [[17], [10, 12], [20, 20], [7, 13], [], [7, 11]]

result = generate_ft_sgns_samples(text, window_size, vocab_size, ns_rate, token2subwords)

print(repr(result))

**3.4.4**

In [ ]:
def update_ft_weights(center_embeddings, context_embeddings, center_subwords, context_word, label, learning_rate):
    """
    center_embeddings - VocabSize x EmbSize
    context_embeddings - VocabSize x EmbSize
    center_subwords - list of ints - list of identifiers of n-grams contained in center word
    context_word - int - identifier of context word
    label - 1 if context_word is real, 0 if it is negative
    learning_rate - float > 0 - size of gradient step
    """
    #заменил для истории на подход от Дани Крутовкий, у меня было чуть длинее, за счет того,
    #что я забыл, что для выбора нескольких значений сразу можно слайс подавать
    w_mean = center_embeddings[center_subwords].mean(-2)
    d = context_embeddings[context_word]

    sigma = 1/(1+np.exp(-w_mean@d))
    grad_w = (sigma-label) * d
    grad_d = (sigma-label) * w_mean
    
    context_embeddings[context_word] -= learning_rate * grad_d
    center_embeddings[center_subwords] -= learning_rate * grad_w / len(center_subwords)

**3.4.5**

In [ ]:
import sys
import ast
import numpy as np
import scipy.sparse


def read_array():
    return ast.literal_eval(sys.stdin.readline())

def write_array(arr):
    print(repr(arr.tolist()))

def generate_coocurrence_matrix(texts, vocab_size):
    """
    texts - list of lists of ints - i-th sublist contains identifiers of tokens in i-th document
    vocab_size - int - size of vocabulary
    returns scipy.sparse.dok_matrix
    """
    co_matrix = scipy.sparse.dok_matrix((vocab_size, vocab_size), dtype=np.float32)
    for doc in text:
        tokens = tuple(set(doc))
        for i in tokens:
            co_matrix[i,tokens] += 1   
    co_matrix.setdiag(0)
    return co_matrix

text = [[0, 2, 2, 2, 0, 0], [1, 1, 2, 1, 1], [2, 2, 1, 1]]
vocab_size = 3

result = generate_coocurrence_matrix(text, vocab_size)

write_array(result.toarray())

In [ ]:
import sys
import ast
import numpy as np
import scipy.sparse


def read_array():
    return ast.literal_eval(sys.stdin.readline())

def write_array(arr):
    print(repr(arr.tolist()))

def generate_coocurrence_matrix(texts, vocab_size):
    """
    texts - list of lists of ints - i-th sublist contains identifiers of tokens in i-th document
    vocab_size - int - size of vocabulary
    returns scipy.sparse.dok_matrix
    """
    co_matrix = scipy.sparse.dok_matrix((vocab_size, vocab_size), dtype=np.float32)
    for doc in text:
        tokens = tuple(set(doc))
        mesh = np.array(np.meshgrid(tokens, tokens)).reshape(2,-1)
        co_matrix[mesh[0],mesh[1]] += 1
    co_matrix.setdiag(0)
    return co_matrix

text = [[0, 2, 2, 2, 0, 0], [1, 1, 2, 1, 1], [2, 2, 1, 1]]
vocab_size = 3

result = generate_coocurrence_matrix(text, vocab_size)

write_array(result.toarray())

**3.4.6**

In [ ]:
def weight(x, max_x, alpha):
    mask = (x <= max_x)
    out = np.ones(x.shape)
    out[mask] = (x[mask]/max_x)**alpha
    return out

def update_glove_weights(x, w, d, alpha, max_x, learning_rate):
    """
    x - square integer matrix VocabSize x VocabSize - coocurrence matrix
    w - VocabSize x EmbSize - first word vectors
    d - VocabSize x EmbSize - second word vectors
    alpha - float - power in weight smoothing function f
    max_x - int - maximum coocurrence count in weight smoothing function f
    learning_rate - positive float - size of gradient step
    """
    f = weight(x, max_x, alpha)
    
    pre = f*2*(w@d.T - np.log1p(x))
    grad_w = pre @ d
    grad_d = pre.T @ w
    
    w -= learning_rate * grad_w
    d -= learning_rate * grad_d

**3.4.7**

In [ ]:
import sys
import ast
import numpy as np


def parse_array(s):
    return np.array(ast.literal_eval(s))

def read_array():
    return parse_array(sys.stdin.readline())

def write_array(arr):
    print(repr(arr.tolist()))

def get_nearest(embeddings, query_word_id, get_n):
    """
    embeddings - VocabSize x EmbSize - word embeddings
    query_word_id - integer - id of query word to find most similar to
    get_n - integer - number of most similar words to retrieve

    returns list of `get_n` tuples (word_id, similarity) sorted by descending order of similarity value
    """
    norm_emb = (embeddings.T / (np.linalg.norm(embeddings,axis=-1) + 1e-9)).T
    sims = -np.linalg.norm(norm_emb-norm_emb[query_word_id], axis=-1)
    return [[arg, sims[arg]] for arg in np.argsort(-sims)[:get_n]]


embeddings = np.array([[0.7299015792584768, 0.2915364327741303, 0.5307571134639943, 0.3101345732086396, 
                        0.8327085262119636, 0.39018382511314353, 0.678094726221033, 0.12372148102696612, 
                        0.5966533433209616], [0.5411155947267721, 0.046791742239819856, 0.5358832195593092, 
                        0.09894162419462038, 0.6350557173679914, 0.15126161842015717, 0.11375720216711405, 
                        0.46954553941325416, 0.8281402097264261], [0.5323869209381028, 0.2005012376766715,
                        0.5925043884236925, 0.4621530177251649, 0.3886830034303448, 0.6403738184472031, 
                        0.23320289120963578, 0.43574647265888766, 0.5305633832484254]])
query_word_id = 0
get_n = 8

result = get_nearest(embeddings, query_word_id, get_n)

write_array(np.array(result))

**3.6.2**

In [ ]:
import numpy as np
kernel = np.array((-0.5,0,0.5))
signal = np.array((1,1,2,3,3,3,2,1,1), dtype=np.float32)
" ".join(map(str,([signal[i:i+len(kernel)]@kernel for i in range(len(signal) - len(kernel)+1)])))

**3.6.4**

In [ ]:
import numpy as np
kernel = np.array([[1,1,0],
                   [0,1,1]], dtype=np.float32) #Kernel(InChannels,K)
X = np.array([[1,0],
              [1,1],
              [0,0],
              [0,1],
              [1,0]],dtype=np.float32) #X(InLen,InChannels)
bias = 0
out = [bias + X[i:i+kernel.shape[1],:].flatten()@kernel.T.flatten() for i in range(X.shape[0] - kernel.shape[1]+1)]
" ".join(map(str,out))

In [ ]:
import numpy as np
kernel = np.array([[1,1,0],
                   [0,1,1]], dtype=np.float32) #Kernel(InChannels,K)
X = np.array([[1,0],
              [1,1],
              [0,0],
              [0,1],
              [1,0]],dtype=np.float32) #X(InLen,InChannels)
bias = 0
out = [bias + np.sum(X[i:i+kernel.shape[1],:]*kernel.T) for i in range(X.shape[0] - kernel.shape[1]+1)]
" ".join(map(str,out))

**3.6.5**

In [ ]:
import numpy as np
kernel = np.array([[[1,1,0],
                   [0,1,1]],
                  [[1,0,0],
                   [0,0,1]]], dtype=np.float32) #Kernel(OutChannels,InChannels,K)
X = np.array([[1,0],
              [1,1],
              [0,0],
              [0,1],
              [1,0]],dtype=np.float32) #X(InLen,InChannels)
bias = 0
out = np.array([bias + np.sum(X[i:i+kernel.shape[2],:]*kernel.transpose((0,2,1)), axis=(1,2)) \
                for i in range(X.shape[0] - kernel.shape[2]+1)]).reshape(-1,kernel.shape[0])
print("\n".join(map(lambda row: str(row)[1:-1],out)))

In [ ]:
#решение от Роман Григоров
#Обновил на более оптимальное решение.
#Разик делаем матричное умножение, потом суммы диагоналей с шагом считаем
import numpy as np

kernel = np.array([[[1,1,0],
                   [0,1,1]],
                  [[1,0,0],
                   [0,0,1]]], dtype=np.float32) #Kernel(OutChannels,InChannels,K)
X = np.array([[1,0],
              [1,1],
              [0,0],
              [0,1],
              [1,0]],dtype=np.float32) #X(InLen,InChannels)

x_kernel_product = X @ kernel
y = np.array([np.trace(x_kernel_product, pos, 2) for pos in range(X.shape[0] - kernel.shape[2]+1)])
y

**3.7.Разбор видео**

In [ ]:
import numpy as np

kern_size = 3
sequence_len = 7
in_channel = 4

x = np.array([[1,2,3,4,5,6,7],
              [10,20,30,40,50,60,70],
              [1,2,3,4,5,6,7],
              [1,2,3,4,5,6,7]])
chunks = []
chunk_size = sequence_len - kern_size + 1
for offset in range(kern_size):
    chunks.append(x[:,offset:offset+chunk_size])
chunks

**3.8.1**

In [ ]:
def apply_convolution(data, kernel, bias):
    """
    data - InLen x InChannels
    kernel - OutChannels x InChannels x KernelSize
    bias - OutChannels

    returns OutLen x OutChannels
    """
    return np.array([bias + np.sum(data[i:i+kernel.shape[2],:]*kernel.transpose((0,2,1)), axis=(1,2)) \
                for i in range(data.shape[0] - kernel.shape[2]+1)]).reshape(-1,kernel.shape[0])


**3.8.2**

In [ ]:
def calculate_kernel_grad(x, y, kernel, bias):
    """
    x - InLen x InChannels
    y - OutLen x OutChannels
    kernel - OutChannels x InChannels x KernelSize
    bias - OutChannels

    returns OutChannels x InChannels x KernelSize
    """
    k_ones = np.ones(kernel.shape).transpose((0,2,1))
    return np.sum([x[i:i+k_ones.shape[1],:]*k_ones for i in range(x.shape[0] - k_ones.shape[1]+1)],axis=0).transpose((0,2,1))

In [ ]:
def calculate_kernel_grad(x, y, kernel, bias):
    """
    x - InLen x InChannels
    y - OutLen x OutChannels
    kernel - OutChannels x InChannels x KernelSize
    bias - OutChannels

    returns OutChannels x InChannels x KernelSize
    """
    return np.repeat(np.sum([x[i:i+kernel.shape[2],:] for i in range(x.shape[0] - kernel.shape[2]+1)],axis=0)\
          .transpose((1,0))[np.newaxis,:,:],kernel.shape[0],axis=0)

**3.8.3**

In [ ]:
def calculate_conv_x_grad(x, y, kernel, bias):
    """
    x - InLen x InChannels
    y - OutLen x OutChannels
    kernel - OutChannels x InChannels x KernelSize
    bias - OutChannels

    returns InLen x InChannels
    """
    stkernel = np.sum(kernel, axis=0).T
    x_grad = np.zeros(x.shape)
    for i in range(x.shape[0] - kernel.shape[2]+1):
        x_grad[i:i+kernel.shape[2],:] += stkernel
    return x_grad

**3.8.4**

In [ ]:
def calculate_receptive_field(layers):
    """
    layers - list of LayerInfo

    returns int - receptive field size
    """
    return 1 + sum(map(lambda x: (x.kernel_size-1)*x.dilation, layers))

**4.4.4**

In [ ]:
import torch
import torch.nn.functional as F

W = torch.tensor([[1.,0.,2.],
                  [0.,1.,3.],
                  [1.,3.,0.],
                  [0.,0.,0.]]) #InLen x EmbSize
query = torch.tensor([0.,0.,1.])

#насколько каждое слово (токен) релевантно (похоже на) слову-запросу (на выходе каждого слова скаляр, 
#чем он больше, тем выше релевантность данного слова)
unnorm_scores = W @ query #InLen
print(unnorm_scores)

#нормируем через softmax
att_scores = F.softmax(unnorm_scores, dim=-1) #InLen
print(att_scores)

#каждое измерение эмбеддинга сжимаем до скаляра на основе полученной нормированной релевантности (значимости)
result = W.T @ att_scores #EmbSize
print(result)

**4.5.2**

In [ ]:
def print_matrix(matrix):
    print('\n'.join([' '.join(map(lambda x: '{:.2f}'.format(x), row)) for row in list(matrix)]))

In [ ]:
import torch

W = torch.tensor([[1.,0.],
                  [0.,1.],
                  [1.,1.],
                  [0.,0.]]) #InLen x EmbSize
logits = W @ W.T #InLen x InLen
print_matrix(logits)

**4.5.3**

In [ ]:
import torch
import torch.nn.functional as F
att_scores = F.softmax(logits, dim=-1)
print(att_scores)
result = att_scores @ W
print_matrix(result)

**4.5.4**

In [ ]:
import torch
import torch.nn.functional as F

Input = torch.tensor([[1.,0.],
                      [0.,1.],
                      [1.,1.],
                      [0.,0.]]) #InLen x EmbSize
ProjK = torch.tensor([[1.,0.],
                      [0.,0.]]) #EmbSize x EmbSize
ProjQ = torch.tensor([[0.,0.],
                      [1.,0.]]) #EmbSize x EmbSize
ProjV = torch.tensor([[1.,0.],
                      [0.,1.]]) #EmbSize x EmbSize
BiasK = BiasQ = BiasV = torch.tensor([0.,0.])

Keys = Input @ ProjK + BiasK
Queries = Input @ ProjQ + BiasQ
Values = Input @ ProjV + BiasV

Logits = Queries @ Keys.T #InLen x InLen
AttScores = F.softmax(Logits, dim=-1)
Result = AttScores @ Values
print_matrix(Result)

**4.5.5**

In [ ]:
import torch
import torch.nn.functional as F

Input = torch.tensor([[1.,0.],
                      [0.,1.],
                      [1.,1.],
                      [0.,0.]]) #InLen x EmbSize

Proj1K = torch.tensor([[1.,0.],
                       [0.,0.]]) #
Proj1Q = torch.tensor([[0.,1.],
                       [1.,0.]]) #
Proj1V = torch.tensor([[1.],
                       [0.]]) #

Proj2K = torch.tensor([[0.,0.],
                       [1.,0.]]) #
Proj2Q = torch.tensor([[1.,1.],
                       [1.,1.]]) #
Proj2V = torch.tensor([[0.],
                       [1.]]) #

BiasK = BiasQ = torch.tensor([0.,0.])
BiasV = torch.tensor([0.])

Keys1 = Input @ Proj1K + BiasK
print(Keys1)
Queries1 = Input @ Proj1Q + BiasQ
print(Queries1)
Values1 = Input @ Proj1V + BiasV
print(Values1)

Keys2 = Input @ Proj2K + BiasK
Queries2 = Input @ Proj2Q + BiasQ
Values2 = Input @ Proj2V + BiasV

Logits1 = Queries1 @ Keys1.T #InLen x InLen
print(Logits1)
Logits2 = Queries2 @ Keys2.T #InLen x InLen

AttScores1 = F.softmax(Logits1, dim=-1)
AttScores2 = F.softmax(Logits2, dim=-1)

Result1 = AttScores1 @ Values1
print(Result1)
Result2 = AttScores2 @ Values2
Result = torch.cat([Result1, Result2], dim=1)
print_matrix(Result)

**4.7.1**

In [ ]:
def max_pooling(features, kernel_size):
    """
    features - InLen x EmbSize - features of elements of input sequence
    kernel_size - positive integer - size of sliding window

    returns tuple of two matrices of shape OutLen x EmbSize:
         - output features (main result)
         - relative indices of maximum elements for each position of sliding window
    """
    return np.stack((np.max(features[i:i+kernel_size],axis=0) for i in range(features.shape[0] - kernel_size + 1))),\
           np.stack((np.argmax(features[i:i+kernel_size],axis=0) for i in range(features.shape[0] - kernel_size + 1)))

In [ ]:
#из лучших решений (не мое)
'''
Коммент:
1. Создаём матрицу по которой посчитанные в один чих max, argmax будут нашими ответами
2. Делаем чих
PS: Если столько памяти нет, то добавляем max или argmax в первую строчку решения
PS: На pytorch можно сделать через expand и вообще не пострадать по памяти
'''
def max_pooling(features, kernel_size):
    """
    features - InLen x EmbSize - features of elements of input sequence
    kernel_size - positive integer - size of sliding window

    returns tuple of two matrices of shape OutLen x EmbSize:
         - output features (main result)
         - relative indices of maximum elements for each position of sliding window
    """

    features_stacked = np.stack((features[i:i+kernel_size, ] for i in range(1 + features.shape[0] - kernel_size)))
    result = np.max(features_stacked, axis=1)
    indices = np.argmax(features_stacked, axis=1)
    return result, indices

**4.7.2**

In [ ]:
import sys
import ast
import numpy as np


def parse_array(s):
    return np.array(ast.literal_eval(s))

def read_array():
    return parse_array(sys.stdin.readline())

def write_array(arr):
    print(repr(arr.tolist()))


def max_pooling_dldfeatures(features, kernel_size, indices, dldout):
    """
    features - InLen x EmbSize - features of elements of input sequence
    kernel_size - positive integer - size of sliding window
    indices - OutLen x EmbSize - relative indices of maximum elements for each window position
    dldout - OutLen x EmbSize - partial derivative of loss function with respect to outputs of max_pooling layer

    returns InLen x EmbSize
    """
    out = np.zeros((features.shape))
    for i in range(out.shape[0] - kernel_size + 1):
        indixes = indices[i] * out.shape[1] + np.arange(out.shape[1])
        out[i:i+kernel_size].flat[indixes] += dldout[i]
    return out


features = np.array([[-1.2420542766989977, -0.045100789663994285, 1.858151857421511, 0.10732741246325356], 
                     [-1.480497780371414, -0.12486054931133332, -0.18422425981847368, -1.4228130362490647], 
                     [-0.8417536968892625, 0.9802583655274091, -0.18413492661665792, -1.5582607186399924], 
                     [1.325799250424393, 0.08149768959330334, -1.454876921308986, 0.1408031456023352], 
                     [0.1637602967235608, -0.21250114632967532, 0.8362859721448469, 0.717774697701287], 
                     [-0.7641399532198978, -2.112568530488304, 0.20121440705964902, 0.015624280892385661], 
                     [1.3862200103422582, 0.6508694196448389, -1.162417318743681, 1.5202488401790915], 
                     [1.3947418297193952, -1.013483406336198, -2.0608332074129545, -1.733019236247151], 
                     [1.0932612618870112, 0.8071262618398916, 0.15924519176972282, -0.6885825807454318]])
kernel_size = 6
indices = np.array([[3.0, 2.0, 0.0, 4.0], 
                    [5.0, 1.0, 3.0, 5.0], 
                    [5.0, 0.0, 2.0, 4.0], 
                    [4.0, 5.0, 1.0, 3.0]]).astype('uint32')
dldout = np.array([[-0.0763791951131031, -0.8729161683329371, 0.7454337173675266, -2.508470377801969], 
                   [-0.9080189656976042, 0.6952579391985969, 0.2829942797947518, 0.35396918585149195], 
                   [0.339009358836277, -1.9496823733556254, -0.11017174942549533, 1.4591363247582954], 
                   [-1.1161622731011638, -2.371055190136431, -1.174384738333498, 0.4672192180206214]])

dldfeatures = max_pooling_dldfeatures(features, kernel_size, indices, dldout)

dldfeatures

In [ ]:
#более интересное решение
def max_pooling_dldfeatures(features, kernel_size, indices, dldout):
    """
    features - InLen x EmbSize - features of elements of input sequence
    kernel_size - positive integer - size of sliding window
    indices - OutLen x EmbSize - relative indices of maximum elements for each window position
    dldout - OutLen x EmbSize - partial derivative of loss function with respect to outputs of max_pooling layer

    returns InLen x EmbSize
    """
    result = np.zeros(features.shape)
    for i in range(features.shape[0]-kernel_size+1):
        result[indices[i, :]+i, range(features.shape[1])] += dldout[i,:]
    return result

In [ ]:
#и еще более интересное решение
def max_pooling_dldfeatures(features, kernel_size, indices, dldout):
    """
    features - InLen x EmbSize - features of elements of input sequence
    kernel_size - positive integer - size of sliding window
    indices - OutLen x EmbSize - relative indices of maximum elements for each window position
    dldout - OutLen x EmbSize - partial derivative of loss function with respect to outputs of max_pooling layer

    returns InLen x EmbSize
    """
    doutdfeatures = np.zeros(features.shape)
    out_len = indices.shape[0]
    for i in range(kernel_size):
        doutdfeatures[i:i+out_len] += (indices == i) * dldout
        
    return doutdfeatures


**4.7.3**

In [ ]:
def softmax(x):
    """
    x - vector of n elements - input

    returns vector of n elements - softmax output
    """
    return np.exp(x)/np.sum(np.exp(x))

**4.7.4**

In [10]:
import sys
import ast
import numpy as np


def parse_array(s):
    return np.array(ast.literal_eval(s))

def read_array():
    return parse_array(sys.stdin.readline())

def write_array(arr):
    print(repr(arr.tolist()))


def dsoftmax_dx(x):
    """
    x - vector of n elements - input

    returns matrix n x n
    """
    return np.exp(x)/np.sum(np.exp(x)) * (np.diag(np.ones(x.shape[0])) - (np.exp(x)/np.sum(np.exp(x)))).T


x = np.array([-0.36170314084137395, 1.531638983431016, -1.7131284538840788, -0.9027503682845508, 
              -0.8591376176087115, 0.16481576122888014, 0.2286590883015934, 0.4686776665093307, 
              -0.4880318948728026, 0.13865483857501165, -1.0740873577508447, -1.1693607815929845, 
              0.6388250392697977])

result = dsoftmax_dx(x)

write_array(result)

**4.7.5**

In [41]:
import sys
import ast
import numpy as np


def parse_array(s):
    return np.array(ast.literal_eval(s))

def read_array():
    return parse_array(sys.stdin.readline())

def write_array(arr):
    print(repr(arr.tolist()))


def attention(features, query):
    """
    features - InLen x EmbSize - features of elements of input sequence
    query - EmbSize - features of query object

    returns vector of size EmbSize - features, aggregated according to the query
    """
    #насколько каждое слово (токен) релевантно (похоже на) слову-запросу (на выходе каждого слова скаляр, 
    #чем он больше, тем выше релевантность данного слова)
    unnorm_scores = features @ query #InLen

    #нормируем через softmax
    att_scores = np.exp(unnorm_scores)/np.sum(np.exp(unnorm_scores), axis=-1) #InLen

    #каждое измерение эмбеддинга сжимаем до скаляра на основе полученной нормированной релевантности (значимости)
    return features.T @ att_scores #EmbSize


features = np.array([[0.3504305689198156, 0.871844425624726, 0.29345316540775357, 0.49159320438393916, 0.16391992930609034, 0.24589641847050037, 0.34921020303336925, 0.09968814035867879, 0.8652385667745919, 0.00906484385602968, 0.6134586521086117, 0.08104312584086149, 0.643129733435556, 0.6610968673257929, 0.6825169003800382], [0.005641686042561211, 0.3397733866278605, 0.4408793722092307, 0.6618752692611525, 0.4192615374283991, 0.6718589897811911, 0.23503584107912667, 0.9972834040264165, 0.6907780153811639, 0.5160598448726361, 0.4200243418824855, 0.7745997472321381, 0.9124177261957108, 0.627661131744206, 0.7792319239076758], [0.44947044223343224, 0.39851993627332394, 0.27645205987950927, 0.3360502940952873, 0.20207394761469466, 0.27730469648938627, 0.9647449489128369, 0.38480306917172535, 0.7014748335636187, 0.5616919724157547, 0.3082991954077743, 0.43320540280287834, 0.7682716834674514, 0.04669826413239708, 0.7975639937877288], [0.3529089999231677, 0.5085801437940869, 0.6686697864089949, 0.9579051761714787, 0.14893344048972235, 0.6504801381978066, 0.6554087909852483, 0.2182290972559655, 0.8874805349214916, 0.8549111586624765, 0.2256959432511686, 0.7090739886051667, 0.9215898392742404, 0.8361038069049278, 0.9807575571901945], [0.93096165894251, 0.21204415175966806, 0.005393301311816812, 0.6868163541395496, 0.17651743121260177, 0.4276211882347165, 0.7172630747046246, 0.6222321154458413, 0.782866044726867, 0.9401403417712956, 0.6009251310321828, 0.7689712777389628, 0.011137370287858661, 0.6750220130270511, 0.3656918897133191], [0.344423832576648, 0.5200078573131781, 0.08090528060543856, 0.6187002344784092, 0.24428489996011238, 0.18400539459399, 0.40308101020726217, 0.19255989698913867, 0.8010944590934469, 0.20324438899818598, 0.4927144298170735, 0.03783988662477278, 0.7705093963091103, 0.2520865403496373, 0.40266725180440743], [0.6681310493060861, 0.2801674372250399, 0.6224648405839522, 0.6287746784150413, 0.864080498689899, 0.23833127705610258, 0.20311743810136906, 0.6646132937899738, 0.23575457417289924, 0.1869625695994146, 0.7712148738157554, 0.15237041670323637, 0.2763150373902683, 0.46500408886101585, 0.991468614310106], [0.47955269815875845, 0.18371674117676162, 0.4749895427072034, 0.5127159626377625, 0.14327300286458633, 0.5921086963579639, 0.21467664382766927, 0.08984875049424312, 0.5619088573772313, 0.6324525220346037, 0.65145500723789, 0.5118736033583858, 0.3791794826772541, 0.7062193547285907, 0.12888775429739185], [0.8936198110390413, 0.1499351596848777, 0.23230300209801535, 0.6275970485906217, 0.14179412142521963, 0.44590423506527455, 0.6398118989481705, 0.44025473834142315, 0.9690917160909921, 0.7329911430579539, 0.4723409208689966, 0.30051845327308735, 0.6517065287372249, 0.11682964074366375, 0.3356912564688531], [0.5819483213886298, 0.12715730125007862, 0.6624081011547434, 0.27210122174739293, 0.3321414469174103, 0.6738692045564213, 0.49979407643990537, 0.923095453187033, 0.8688108133354637, 0.29672803800781, 0.8422355709858387, 0.22967466587429908, 0.48606633908371566, 0.3302629931498261, 0.9271715208940098], [0.814229644181095, 0.6269508015754929, 0.19067116118180638, 0.6597416333912787, 0.3042396495694798, 0.5349586078017191, 0.9889297007928726, 0.5059647631701082, 0.6586214714303461, 0.19972197385065704, 0.730120041302739, 0.9254129585548022, 0.7774768791337286, 0.5880525770183761, 0.4404909426586451], [0.8393608070151912, 0.551470751477307, 0.3776646281929925, 0.7403545806778788, 0.01464073752506101, 0.49682079457661743, 0.12829037985166736, 0.8323709714882789, 0.4861583628986299, 0.10966510942571872, 0.36384711262095637, 0.008343156485128067, 0.05481969871494197, 0.11036480291979456, 0.3495717657917], [0.575668909069271, 0.1948209406820347, 0.5066632418120769, 0.5610866065811511, 0.7503051258152065, 0.20250301475454058, 0.9387177222181186, 0.4214964558859865, 0.2441688535705906, 0.2852282954051667, 0.7185375048873539, 0.09961745251862686, 0.507873295740294, 0.9713796833363287, 0.8218946227484244], [0.14519733396011691, 0.07264015089790021, 0.7254237309701331, 0.7437297525624584, 0.3465971472185204, 0.6489212261982703, 0.2152569561085349, 0.6476151760429897, 0.2045187871395916, 0.9599712380254137, 0.28554199184758966, 0.7701922251424572, 0.7095119328780166, 0.7579558453415812, 0.4251898428876446]])
query = np.array([0.30760147020407946, 0.1528992448227442, 0.9387231083505163, 0.12201982125460176, 0.3159744925438269, 0.555332538642272, 0.8654043562058316, 0.5523485724329922, 0.6405492495162189, 0.8421217300945876, 0.03415012932624606, 0.0914780538557024, 0.745151636966557, 0.9885343010237021, 0.02289480154711454])

result = attention(features, query)

write_array(result)

**4.7.6**

In [58]:
import sys
import ast
import numpy as np


def parse_array(s):
    return np.array(ast.literal_eval(s))

def read_array():
    return parse_array(sys.stdin.readline())

def write_array(arr):
    print(repr(arr.tolist()))


def self_attention(features, proj_k, bias_k, proj_q, bias_q, proj_v, bias_v):
    """
    features - InLen x EmbSize - features of elements of input sequence
    proj_k - EmbSize x EmbSize - projection matrix to make keys from features
    bias_k - EmbSize - bias vector to make keys from features
    proj_q - EmbSize x EmbSize - projection matrix to make queries from features
    bias_q - EmbSize - bias vector to make queries from features
    proj_v - EmbSize x EmbSize - projection matrix to make values from features
    bias_v - EmbSize - bias vector to make values from features

    returns InLen x EmbSize
    """
    keys = features @ proj_k + bias_k 
    queries = features @ proj_q + bias_q
    values = features @ proj_v + bias_v

    logits = queries @ keys.T #InLen x InLen
    att_scores = (np.exp(logits.T)/np.sum(np.exp(logits.T), axis=0)).T #InLen x InLen
    
    return att_scores @ values


features = np.array([[0.5175129778200084, 0.13021330700949507], 
                     [-0.3578609445921744, -0.07768163060380659], 
                     [-0.046577477754636734, -0.12288550821838619], 
                     [0.4424092505449793, -1.431399548551344], 
                     [0.753992222548331, -1.1210257338970167], 
                     [1.6736061037504428, -1.9789731491226337], 
                     [-1.4985152255486565, -1.6614802556117283], 
                     [-0.610065708073959, -0.8475335063027695], 
                     [-0.1657640783522184, -1.7079776825852762], 
                     [0.7857341373616981, 0.2956255012408635], 
                     [-0.49243028413686984, 0.01065675311085114], 
                     [0.20598401523943788, -0.6339670563637549], 
                     [-0.15698934123474126, 0.9516567843056503], 
                     [-0.08965595798444444, 0.9923765422389716], 
                     [-0.9649404480809814, -0.6203623955866846]])
proj_k = np.array([[-0.4777373377067222, 1.384780896738418], 
                   [1.5537173245233542, -1.6151073640132454]])
bias_k = np.array([0.34068677065672126, -1.7225350645946451])

proj_q = np.array([[1.667192089239799, 0.9072106203091014], 
                   [1.0017863742909645, -0.8578876449703756]])
bias_q = np.array([-0.3811843050970959, -0.15922065479353645])

proj_v = np.array([[2.396375885002737, 0.23995979796695774], 
                   [0.21631803999882093, -0.6475173781192963]])
bias_v = np.array([1.448781271879823, -0.7144164516316529])

result = self_attention(features, proj_k, bias_k, proj_q, bias_q, proj_v, bias_v)

result

In [24]:
keys = features @ proj_k + bias_k
keys

In [25]:
queries = features @ proj_q + bias_q
queries

In [26]:
values = features @ proj_v + bias_v
values

In [28]:
logits = queries @ keys.T #InLen x InLen
logits.shape

In [35]:
att_scores = np.exp(logits)/np.sum(np.exp(logits), axis=1) #InLen
sum(att_scores[:,0])

In [46]:
logits = np.array([[1.,1.,1.],
                   [-999., 1.,1.],
                   [-999., -9999.,1.]])
logits

In [57]:
att_scores = (np.exp(logits.T)/np.sum(np.exp(logits.T), axis=0)).T #InLen
att_scores

In [ ]:
import torch
import torch.nn.functional as F

Input = torch.tensor([[1.,0.],
                      [0.,1.],
                      [1.,1.],
                      [0.,0.]]) #InLen x EmbSize
ProjK = torch.tensor([[1.,0.],
                      [0.,0.]]) #EmbSize x EmbSize
ProjQ = torch.tensor([[0.,0.],
                      [1.,0.]]) #EmbSize x EmbSize
ProjV = torch.tensor([[1.,0.],
                      [0.,1.]]) #EmbSize x EmbSize
BiasK = BiasQ = BiasV = torch.tensor([0.,0.])

Keys = Input @ ProjK + BiasK
Queries = Input @ ProjQ + BiasQ
Values = Input @ ProjV + BiasV

Logits = Queries @ Keys.T #InLen x InLen
AttScores = F.softmax(Logits, dim=-1)
Result = AttScores @ Values
print_matrix(Result)